# miRNA-mRNA target prediction


### 1. Problem definition

This project aims to <b>predict if an input pair of sequences (miRNA and mRNA) will interact</b>; given that the molecules interact, they constitute a pair, whereas if they do not interact, they are defined as non-pair.

Since the mechanisms underlying the targeting process can differ between organisms, this study is <b>focused on the A. thaliana organism</b>. This is a model organism, meaning that the results obtained can be extrapolated to other plants; furthermore, they can be used to understand human diseases due to the conservation of protein function, conservation of cellular processes, and the high percentage of genes shared between both species [31, 32].

Considering the above, this problem can be shaped as a <b>binary classification problem</b>, where 0 means non-pair and 1 represents a pair.

The data proposed to train a DNN able to distinguish between pair and non-pair RNA sequences consists of curated interactions that are publicly available and reported in the literature [14].

Although such miRNA-mRNA interacting pairs can be used to train the network, they represent only positive examples that were tested experimentally either in vivo or in vitra. This implies that the <b>available datasets are unbalanced</b>, and denotes the need to incorporate negative data before proceeding.

### 2. Dataset selection


### 3. Success measures

### 4. Evaluation protocols

### 5. Data preparation

### 6. Model beating the baseline

### 7. Model overfitting


### 8. Regularization and hyperparameter tuning

In [1]:
import tensorflow as tf

2023-08-24 12:46:13.126829: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### References

[1] A. Pla, X. Zhong, and S. Rayner, “miRAW: A deep learning-based approach to predict microRNA targets by analyzing whole microRNA transcripts,” PLOS Computational Biology, vol. 14, no. 7, p. e1006185, Jul. 2018, doi: 10.1371/journal.pcbi.1006185.

[2] J. O’Brien, H. Hayder, Y. Zayed, and C. Peng, “Overview of MicroRNA Biogenesis, Mechanisms of Actions, and Circulation,” Frontiers in Endocrinology, vol. 9, 2018, Accessed: May 04, 2023. [Online]. Available: https://www.frontiersin.org/articles/10.3389/fendo.2018.00402

[3] A. Quillet et al., “Improving Bioinformatics Prediction of microRNA Targets by Ranks Aggregation,” Frontiers in Genetics, vol. 10, 2020, Accessed: May 04, 2023. [Online]. Available: https://www.frontiersin.org/articles/10.3389/fgene.2019.01330

[4] H. Nakayashiki, ‘RNA silencing in fungi: Mechanisms and applications’, FEBS Letters, vol. 579, no. 26, pp. 5950–5957, Oct. 2005, doi: 10.1016/j.febslet.2005.08.016.

[5] T. Kakati, D. K. Bhattacharyya, J. K. Kalita, and T. M. Norden-Krichmar, ‘DEGnext: classification of differentially expressed genes from RNA-seq data using a convolutional neural network with transfer learning’, BMC Bioinformatics, vol. 23, no. 1, p. 17, Jan. 2022, doi: 10.1186/s12859-021-04527-4.

[6] B. Hanczar, F. Zehraoui, T. Issa, and M. Arles, ‘Biological interpretation of deep neural network for phenotype prediction based on gene expression’, BMC Bioinformatics, vol. 21, no. 1, p. 501, Nov. 2020, doi: 10.1186/s12859-020-03836-4.

[7] D. Urda, J. Montes-Torres, F. Moreno, L. Franco, and J. M. Jerez, ‘Deep Learning to Analyze RNA-Seq Gene Expression Data’, in Advances in Computational Intelligence, I. Rojas, G. Joya, and A. Catala, Eds., in Lecture Notes in Computer Science, vol. 10306. Cham: Springer International Publishing, 2017, pp. 50–59. doi: 10.1007/978-3-319-59147-6_5.

[8] ‘Central Dogma’, Genome.gov, Sep. 14, 2022. https://www.genome.gov/genetics-glossary/Central-Dogma (accessed May 07, 2023).

[9] A. Talukder, W. Zhang, X. Li, and H. Hu, “A deep learning method for miRNA/isomiR target detection,” Sci Rep, vol. 12, no. 1, Art. no. 1, Jun. 2022, doi: 10.1038/s41598-022-14890-8.

[10] O. P. Gupta, P. Sharma, R. K. Gupta, and I. Sharma, “Current status on role of miRNAs during plant–fungus interaction,” Physiological and Molecular Plant Pathology, vol. 85, pp. 1–7, Jan. 2014, doi: 10.1016/j.pmpp.2013.10.002.

[11] E. Marín-González and P. Suárez-López, “‘And yet it moves’: Cell-to-cell and long-distance signaling by plant microRNAs,” Plant Science, vol. 196, pp. 18–30, Nov. 2012, doi: 10.1016/j.plantsci.2012.07.009.

[12] T. Siddika and I. U. Heinemann, “Bringing MicroRNAs to Light: Methods for MicroRNA Quantification and Visualization in Live Cells,” Frontiers in Bioengineering and Biotechnology, vol. 8, 2021, Accessed: Apr. 18, 2023. [Online]. Available: https://www.frontiersin.org/articles/10.3389/fbioe.2020.619583

[13] J. K. W. Lam, M. Y. T. Chow, Y. Zhang, and S. W. S. Leung, “siRNA Versus miRNA as Therapeutics for Gene Silencing,” Mol Ther Nucleic Acids, vol. 4, no. 9, p. e252, Sep. 2015, doi: 10.1038/mtna.2015.23.

[14] “miRTarBase: the experimentally validated microRNA-target interactions database.” https://mirtarbase.cuhk.edu.cn/~miRTarBase/miRTarBase_2022/php/index.php (accessed May 08, 2023).

[15] “Gene Regulation,” Genome.gov, Sep. 14, 2022. https://www.genome.gov/genetics-glossary/Gene-Regulation (accessed May 09, 2023).

[16] C. Stylianopoulou, “Carbohydrates: Regulation of metabolism,” in Encyclopedia of Human Nutrition (Fourth Edition), B. Caballero, Ed., Oxford: Academic Press, 2023, pp. 126–135. doi: 10.1016/B978-0-12-821848-8.00173-6.

[17] L. He and G. J. Hannon, “MicroRNAs: small RNAs with a big role in gene regulation,” Nat Rev Genet, vol. 5, no. 7, Art. no. 7, Jul. 2004, doi: 10.1038/nrg1379.

[18] D. Pradhan, A. Kumar, H. Singh, and U. Agrawal, “Chapter 4 - High-throughput sequencing,” in Data Processing Handbook for Complex Biological Data Sources, G. Misra, Ed., Academic Press, 2019, pp. 39–52. doi: 10.1016/B978-0-12-816548-5.00004-6.

[19] B. Hanczar, F. Zehraoui, T. Issa, and M. Arles, “Biological interpretation of deep neural network for phenotype prediction based on gene expression,” BMC Bioinformatics, vol. 21, no. 1, p. 501, Nov. 2020, doi: 10.1186/s12859-020-03836-4.

[20] A. L. Leitão and F. J. Enguita, “A Structural View of miRNA Biogenesis and Function,” Non-Coding RNA, vol. 8, no. 1, Art. no. 1, Feb. 2022, doi: 10.3390/ncrna8010010.

[21] ‘Gene Expression | Learn Science at Scitable’. https://www.nature.com/scitable/topicpage/gene-expression-14121669/ (accessed May 07, 2023).

[22] W. Guo, Y. Xu, and X. Feng, ‘DeepMetabolism: A Deep Learning System to Predict Phenotype from Genome Sequencing’. arXiv, May 08, 2017. doi: 10.48550/arXiv.1705.03094.

[23] M. Wen, P. Cong, Z. Zhang, H. Lu, and T. Li, ‘DeepMirTar: a deep-learning approach for predicting human miRNA targets’, Bioinformatics, vol. 34, no. 22, pp. 3781–3787, Nov. 2018, doi: 10.1093/bioinformatics/bty424.

[24] X. M. Xu and S. G. Møller, ‘The value of Arabidopsis research in understanding human disease states’, Curr Opin Biotechnol, vol. 22, no. 2, pp. 300–307, Apr. 2011, doi: 10.1016/j.copbio.2010.11.007.

[25] G. P. Way and C. S. Greene, ‘Extracting a Biologically Relevant Latent Space from Cancer Transcriptomes with Variational Autoencoders’. bioRxiv, p. 174474, Aug. 11, 2017. doi: 10.1101/174474.

[26] J. Rocca, ‘Understanding Variational Autoencoders (VAEs)’, Medium, Mar. 21, 2021. https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73 (accessed Jun. 07, 2023).

[27] C. H. Grønbech, M. F. Vording, P. Timshel, C. K. Sønderby, T. H. Pers, and O. Winther, ‘scVAE: Variational auto-encoders for single-cell gene expression data’. bioRxiv, p. 318295, Oct. 02, 2019. doi: 10.1101/318295.

[28] K. Y. Gao, A. Fokoue, H. Luo, A. Iyengar, S. Dey, and P. Zhang, ‘Interpretable Drug Target Prediction Using Deep Neural Representation’, in Proceedings of the Twenty-Seventh International Joint Conference on Artificial Intelligence, Stockholm, Sweden: International Joint Conferences on Artificial Intelligence Organization, Jul. 2018, pp. 3371–3377. doi: 10.24963/ijcai.2018/468.

[29] ‘Arabidopsis thaliana (ID 4) - Genome - NCBI’. https://www.ncbi.nlm.nih.gov/genome/4?genome_assembly_id=380024 (accessed Jul. 02, 2023).

[30] G. B. Or and I. Veksler-Lublinsky, ‘Comprehensive machine-learning-based analysis of microRNA-target interactions reveals variable transferability of interaction rules across species’. bioRxiv, p. 2021.03.28.437385, Mar. 29, 2021. doi: 10.1101/2021.03.28.437385.

[31] ‘Arabidopsis thaliana (ID 4) - Genome - NCBI’. https://www.ncbi.nlm.nih.gov/genome/4?genome_assembly_id=380024 (accessed Jul. 02, 2023).

[32] X. Chen, ‘Small RNAs – secrets and surprises of the genome’, Plant J, vol. 61, no. 6, pp. 941–958, Mar. 2010, doi: 10.1111/j.1365-313X.2009.04089.x.

[33] S. Bandyopadhyay and R. Mitra, ‘TargetMiner: microRNA target prediction with systematic identification of tissue-specific negative examples’, Bioinformatics, vol. 25, no. 20, pp. 2625–2631, Oct. 2009, doi: 10.1093/bioinformatics/btp503.

[34] ‘PmiREN: Plant microRNA Encyclopedia’. https://www.pmiren.com/download (accessed Aug. 04, 2023).

[35] ‘refSeq Accession to Gene Symbol Converter - Genomics Biotools’. https://www.biotools.fr/mouse/refseq_symbol_converter (accessed Aug. 07, 2023).

[36] ‘miRBase - Downloads’. https://mirbase.org/download/ (accessed Aug. 13, 2023).

[37] ‘Genome’, NCBI. https://www.ncbi.nlm.nih.gov/datasets/genome/?taxon=9606 (accessed Aug. 13, 2023).

[38] ‘11968211 - Assembly - NCBI’. https://www.ncbi.nlm.nih.gov/assembly/?term=GCF_000001405 (accessed Aug. 13, 2023).

[39] B. Murcott, R. J. Pawluk, A. V. Protasio, R. Y. Akinmusola, D. Lastik, and V. L. Hunt, ‘stepRNA: Identification of Dicer cleavage signatures and passenger strand lengths in small RNA sequences’, Frontiers in Bioinformatics, vol. 2, 2022, Accessed: Aug. 18, 2023. [Online].
